In [68]:
import numpy as np
import os  # For Saving to Folder
import pandas as pd
import matplotlib.pyplot as plt

import socket
import os as os
import sys as sys
import multiprocessing as mp
from pysam import AlignmentFile

### For Arial Font
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'   # Set the defaul
### Make sure to have the font installed (it is on cluster for Harald)
rcParams['font.sans-serif'] = ['Arial']

socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/y_chrom/"  # The Path on Midway Cluster
    
elif socket_name.startswith("bionc"):
    print("Leipzig Cluster detected!")
    path = "/mnt/archgen/users/hringbauer/git/y_chrom/"
    
else:
    raise RuntimeWarning("Not compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")
print(sys.version)

bionc21
Leipzig Cluster detected!
/mnt/archgen/users/hringbauer/git/y_chrom
CPU Count: 40
3.8.10 (default, Nov 22 2023, 10:22:35) 
[GCC 9.4.0]


In [2]:
def load_counts(path_counts, coerce=True):
    """Load Count file and return Dataframe"""
    df_t = pd.read_csv(path_counts, header=None, delim_whitespace=True)
    df_t.columns = ["snp", "chr", "pos", "ref_all", "alt_all", "drop", "iid", "ref", "alt"]
    
    if coerce:
        for col in ["pos", "ref", "alt"]:
            df_t[col] = pd.to_numeric(df_t[col], errors="coerce")
            
    df_t = df_t.drop(columns="drop")
    return df_t

def load_snp_file_ISOGG(path_snps = "./data/all_snps.csv", 
                    col_pos = 'Build 37 Number', unique=True):
    """Return Dataframe in Eigenstrat Format,
    filtered for biallelic SNPs.
    unique: Whether to keep a """
    df_raw = pd.read_csv(path_snps)
    print(df_raw.columns)
    print(f"Loaded {len(df_raw)} SNPs")

    ### Process the positions
    pos = df_raw[col_pos]
    df_raw["pos"] = pd.to_numeric(pos, errors="coerce")

    idx = ~df_raw["pos"].isna()
    print(f"# Positions available: {np.sum(idx)}")
    df = df_raw[idx].reset_index(drop=True)
    df["pos"]=df["pos"].astype("int")

    idx_bi= (df["Mutation Info"].str.len()==4)
    print(f"# Biallelic SNPs: {np.sum(idx_bi)}")
    df = df[idx_bi].reset_index(drop=True)
    df["ref"] = df["Mutation Info"].str[0]
    df["alt"] = df["Mutation Info"].str[3]
    df["chrom"] = "Y"

    cols = ["Name", "chrom", "pos", "ref", "alt", 
            'Subgroup Name', 'Alternate Names', 'rs numbers']
    df = df[cols]
    df = df.replace(regex=[' ','\n'], value='_')
    ### Sort by position
    df = df.sort_values(by="pos")
    
    ### Keep only SNPs where Ref and Alt Different
    idx_same = (df["ref"]==df["alt"])
    df = df[~idx_same]
    print(f"# Ref & Alt different: {len(df)}")
    
    ### Keep only ACTG
    snps_acceptable = ["A", "C", "T", "G"]
    idx_ref = df["ref"].isin(snps_acceptable)
    idx_alt = df["alt"].isin(snps_acceptable)
    idx_both = idx_ref & idx_alt
    df = df[idx_both]
    print(f"# Ref & Alt ACTG: {len(df)}")
    
    ### Keep Unique Values
    if unique:
        idx_dup = df.duplicated(subset=["pos", "ref", "alt"], keep="first")
        df = df[~idx_dup]
        print(f"# Unique SNP positions: {len(df)}")
    
    ### Remove duplicate Names
    #idx_dup = df.duplicated(subset="Name", keep=False)
    #df = df[~idx_dup]
    #print(f"# Unique Names: {len(df)}")
    return df.copy().reset_index(drop=True)


################################################
### Calling Ys

def ref_alt_count(df_ch, bases=["A", "C", "G", "T"]):
    """Count Ref and Alt alleles in Dataframe df_ch
    with ref, alt, A, C, G, T fields and enter new columns
    ref# and alt#"""
    df_ch["ref#"]=0
    df_ch["alt#"]=0

    for p in bases:
        idx = df_ch["ref"] == p
        df_ch.loc[idx, "ref#"] = df_ch.loc[idx, p]

        idx = df_ch["alt"] == p
        df_ch.loc[idx, "alt#"] = df_ch.loc[idx, p]
    return df_ch

def pulldown_bamtable(path_bam = "", o_file = "",                   
                      bamtable = "/home/pruefer/bin/BamTable",
                      path_bed = "/mnt/archgen/users/hringbauer/git/y_chrom/data/isogg_snps.bed"):
    """Pulldown a BAM at path_bam to o_file using bamtable and the bed a path_bed."""
    !$bamtable -F -A -f $path_bed $path_bam > $o_file
    

def call_y_bam(path_bam="", df=[],
               path_bed = "/mnt/archgen/users/hringbauer/git/y_chrom/data/isogg_snps.bed",
               path_temp="/mnt/archgen/users/hringbauer/git/y_chrom/temp/temp.tsv"):
    """Creates the Call Table from a .bam file"""
    
    ### Create the Pulldown
    pulldown_bamtable(path_bam = path_bam,
                      path_bed = path_bed,
                      o_file = path_temp)

    df1 = pd.read_csv(path_temp, sep="\t", header=None)
    df1.columns = ["chrom", "pos", "A", "C", "G", "T"]
    df2 = pd.merge(df, df1, on=["chrom", "pos"])
    
    ### Coverage Statistics
    cov = df1[["A", "C", "G", "T"]].values
    cov1 = np.sum(cov, axis=1)
    print(f"Average Coverage: {np.sum(cov1)/len(df):.4f}x")
    print(f"#Sites covered: {np.sum(cov1>0)}/{len(df)}")
    
    ### Establish Ref and Alt allele
    df_ch = ref_alt_count(df2, bases=["A", "C", "G", "T"])

    ### Identify Derived    
    idx_der = df_ch["alt#"]>df_ch["ref#"]
    print(f"#Derived Loci: \n{np.sum(idx_der)} / {np.sum(cov1>0)} covered>0")
    
    df_der = df_ch[idx_der].sort_values(by="Subgroup Name").reset_index(drop=True).copy()
    
    return df_ch, df_der 

### Load ISOGG SNPs

In [3]:
df = load_snp_file_ISOGG("./data/all_snps.csv")

Index(['Name', 'Subgroup Name', 'Alternate Names', 'rs numbers',
       'Build 37 Number', 'Build 38 Number', 'Mutation Info'],
      dtype='object')
Loaded 92035 SNPs
# Positions available: 91881
# Biallelic SNPs: 91814
# Ref & Alt different: 91811
# Ref & Alt ACTG: 91806
# Unique SNP positions: 73148


# Run Ancient Brienzi

In [34]:
%%time
df_ch, df_der = call_y_bam(df=df, 
                           path_bam="/mnt/archgen/users/hringbauer/data/brienziYcapture/A55903.bam") #A55903 and A55904

Average Coverage: 15.7615x
#Sites covered: 60398/73148
#Derived Loci: 
1049 / 60398 covered>0
CPU times: user 258 ms, sys: 47.8 ms, total: 306 ms
Wall time: 6.97 s


In [35]:
df_der[-50:]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
999,PF6419,Y,6912992,T,G,R1b1a1b,CTS623,NaN,0,0,16,0,0,16
1000,PF6508,Y,21993844,G,A,R1b1a1b,NaN,NaN,9,0,1,0,1,9
1001,PF6516,Y,22621843,G,T,R1b1a1b,M4435;_Z2129,NaN,0,0,0,3,0,3
1002,PF6421,Y,7220727,A,G,R1b1a1b,F69;_L773;_YSC0000276,NaN,0,0,5,0,0,5
1003,CTS6532,Y,16971648,T,G,R1b1a1b,PF6465,NaN,0,0,3,0,0,3
1004,PF6466,Y,17013730,G,A,R1b1a1b,YSC0000194,NaN,17,0,0,0,0,17
1005,E101,Y,17281258,T,G,R1b1a1b,M12191;_Y1271,NaN,0,0,50,0,0,50
1006,PF6467,Y,17132580,C,T,R1b1a1b,CTS6832,NaN,0,0,0,23,0,23
1007,PF6437,Y,9392948,C,T,R1b1a1b,NaN,NaN,0,0,0,13,0,13
1008,YSC0001293,Y,7073423,G,A,R1b1a1b,CTS894;_PF6420,NaN,6,0,0,0,0,6


In [41]:
df_ch[df_ch["Subgroup Name"]=="R1b1a1b1a1a2a1"]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
39129,Z195,Y,17922066,G,A,R1b1a1b1a1a2a1,S227,NaN,0,0,55,0,55,0


In [51]:
%%time
df_ch, df_der = call_y_bam(df=df, 
                           path_bam="/mnt/archgen/users/hringbauer/data/brienziYcapture/A55904.bam") #A55903 and A55904

Average Coverage: 13.0144x
#Sites covered: 58128/73148
#Derived Loci: 
998 / 58128 covered>0
CPU times: user 233 ms, sys: 16.7 ms, total: 249 ms
Wall time: 5.92 s


### 1)b) Run the second extract

In [51]:
%%time
df_ch, df_der = call_y_bam(df=df, 
                           path_bam="/mnt/archgen/users/hringbauer/data/brienziYcapture/A55904.bam")

Average Coverage: 13.0144x
#Sites covered: 58128/73148
#Derived Loci: 
998 / 58128 covered>0
CPU times: user 231 ms, sys: 55.5 ms, total: 286 ms
Wall time: 5.69 s


In [52]:
df_der[-50:]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
948,PF6427,Y,7863189,G,A,R1b1a1b,L482,NaN,8,0,0,0,0,8
949,PF6508,Y,21993844,G,A,R1b1a1b,NaN,NaN,1,0,0,0,0,1
950,FGC45,Y,17813541,C,T,R1b1a1b,CTS8052;_PF6473,NaN,0,0,0,1,0,1
951,CTS2466,Y,14317555,G,A,R1b1a1b,PF6453,NaN,15,0,0,0,0,15
952,FGC40,Y,19462180,C,T,R1b1a1b,CTS10451;_PF6493,NaN,0,0,0,14,0,14
953,PF6419,Y,6912992,T,G,R1b1a1b,CTS623,NaN,0,0,17,0,0,17
954,PF6478,Y,18117193,C,T,R1b1a1b,CTS8627,NaN,0,0,0,35,0,35
955,CTS10834,Y,22796697,T,C,R1b1a1b,NaN,NaN,0,11,0,0,0,11
956,PF6437,Y,9392948,C,T,R1b1a1b,NaN,NaN,0,0,0,5,0,5
957,CTS8591,Y,18095336,A,C,R1b1a1b,PF6477,NaN,0,7,0,0,0,7


In [56]:
df_ch[df_ch["Subgroup Name"]=="R1b1a1b1a1a2a1"]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
37816,Z195,Y,17922066,G,A,R1b1a1b1a1a2a1,S227,NaN,0,0,10,0,10,0


In [80]:
df_ch[df_ch["pos"]==8502236]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
288,S167,Y,8502236,G,A,R1b1a1b1a,L51;_M412;_PF6536,NaN,54,0,0,0,0,54


### For Replication: Run Brienzi 1240k

In [40]:
%%time
df_ch, df_der = call_y_bam(df=df, 
                           path_bam="/mnt/archgen/users/hringbauer/brienzi/data/capture/A55903.bam") 

Average Coverage: 1.1803x
#Sites covered: 16362/73148
#Derived Loci: 
510 / 16362 covered>0
CPU times: user 148 ms, sys: 54.8 ms, total: 203 ms
Wall time: 1.68 s


In [33]:
df_der[-50:]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
999,PF6419,Y,6912992,T,G,R1b1a1b,CTS623,NaN,0,0,16,0,0,16
1000,PF6508,Y,21993844,G,A,R1b1a1b,NaN,NaN,9,0,1,0,1,9
1001,PF6516,Y,22621843,G,T,R1b1a1b,M4435;_Z2129,NaN,0,0,0,3,0,3
1002,PF6421,Y,7220727,A,G,R1b1a1b,F69;_L773;_YSC0000276,NaN,0,0,5,0,0,5
1003,CTS6532,Y,16971648,T,G,R1b1a1b,PF6465,NaN,0,0,3,0,0,3
1004,PF6466,Y,17013730,G,A,R1b1a1b,YSC0000194,NaN,17,0,0,0,0,17
1005,E101,Y,17281258,T,G,R1b1a1b,M12191;_Y1271,NaN,0,0,50,0,0,50
1006,PF6467,Y,17132580,C,T,R1b1a1b,CTS6832,NaN,0,0,0,23,0,23
1007,PF6437,Y,9392948,C,T,R1b1a1b,NaN,NaN,0,0,0,13,0,13
1008,YSC0001293,Y,7073423,G,A,R1b1a1b,CTS894;_PF6420,NaN,6,0,0,0,0,6


In [31]:
dft = df_ch[df_ch["Subgroup Name"].str[0]==("G")]

a = np.sum(dft["alt#"])
r = np.sum(dft["ref#"])

a/(a+r)

0.0007489390030789714

### Run Shotgun Data

In [38]:
%%time
df_ch, df_der = call_y_bam(df=df, 
                           path_bam="/mnt/archgen/users/hringbauer/data/from_elena/210901_M06210_0360_000000000-JRRVW_JR_B53790_3_CervidsMT_HaraldRingbauerSamples/A52828.bam") 

Average Coverage: 0.0028x
#Sites covered: 205/73148
#Derived Loci: 
5 / 205 covered>0
CPU times: user 85.7 ms, sys: 58.6 ms, total: 144 ms
Wall time: 1.46 s


In [39]:
df_der[-50:]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
0,Z40392,Y,15261255,C,T,BT,NaN,NaN,0,0,0,1,0,1
1,BY1406,Y,18161373,C,T,C2a1a1a,Z30573,NaN,0,0,0,1,0,1
2,FT22041,Y,15434480,G,C,G2a2b2a4b,NaN,NaN,0,1,0,0,0,1
3,PF3496,Y,7702973,T,A,IJK,M2683;_YSC0001318,NaN,1,0,0,0,0,1
4,M5577,Y,2757670,C,T,J1a3b~,PF1413;_V189,NaN,0,0,0,1,0,1


# 2) Run modern Huggler (Ind 1 from 2022)

In [81]:
%%time
p = "/mnt/archgen/users/hringbauer/data/brienziYcapture/huggler.bam"
p2 =  "/mnt/archgen/users/hringbauer/data/brienziYcapture/huggler2.bam"

df_ch, df_der = call_y_bam(df=df, 
                           path_bam=p2)

Average Coverage: 0.5878x
#Sites covered: 1045/73148
#Derived Loci: 
57 / 1045 covered>0
CPU times: user 112 ms, sys: 24.7 ms, total: 137 ms
Wall time: 1.86 s


### Quick quality check
(Whether G SNPs are ancestral)

In [82]:
dft = df_ch[df_ch["Subgroup Name"].str.contains("G")]
s = np.sum(dft["alt#"].values) / np.sum(dft["alt#"].values + dft["ref#"].values)
print(f"Derived Frac: {s:.6f}")

cov = np.mean(dft["alt#"].values + dft["ref#"].values)
print(f"Average Coverage: {cov:.2f}x")

Derived Frac: 0.000228
Average Coverage: 42.23x


In [1]:
0.15 * 2000

300.0

In [83]:
df_der[-50:]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
7,L1121,Y,14496448,G,A,A0-T,NaN,NaN,12,0,0,0,0,12
8,L1120,Y,14496439,G,T,A0-T,NaN,NaN,0,0,0,38,0,38
9,P305,Y,2710154,A,G,A1,NaN,NaN,0,0,18,0,0,18
10,V168,Y,17947672,G,A,A1,NaN,NaN,46,0,0,0,0,46
11,V221,Y,7589303,G,T,A1b,NaN,NaN,0,0,0,35,0,35
12,L1053,Y,18759708,C,A,A1b,NaN,NaN,9,0,0,0,0,9
13,M118,Y,21763965,A,T,A1b1b2b1,NaN,NaN,0,0,0,26,0,26
14,M9379,Y,23244049,G,C,BT,NaN,NaN,0,54,0,0,0,54
15,M42,Y,21866840,A,T,BT,NaN,NaN,0,0,0,29,0,29
16,SRY10831.1,Y,2657176,T,C,BT,NaN,NaN,0,27,0,0,0,27


In [84]:
df_ch[df_ch["Subgroup Name"]=="R1b1a1b1a1a2a"]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#


In [85]:
df_ch[df_ch["Subgroup Name"]=="R1b1a1b1a1a2b3c"]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
400,Z144,Y,14544130,T,G,R1b1a1b1a1a2b3c,NaN,NaN,0,0,32,0,0,32
587,S371,Y,16454755,G,T,R1b1a1b1a1a2b3c,PF6578;_Z145,NaN,0,0,0,57,0,57
1043,Z146,Y,23627289,C,T,R1b1a1b1a1a2b3c,PF6584;_S483,NaN,0,0,0,72,0,72


- R1b1a1b1a   R1b-L51 (old notation R1b1a1a2a1)
- R1b1a1b1a1a2: R1b-P312
 

Trifurcates into:

- R1b1a1b1a1a2a: DF27
- R1b1a1b1a1a2b: U152  
- R1b1a1b1a1a2c1: L21

Huggler:
- R1b1a1b1a1a2b3c
- R1b1a1b1a1a2b: U152

Brienzi:
- R1b1a1b1a1a2a (DF27)
- R1b1a1b1a1a2

### Pre-Processing Steps (1x needed)

In [ ]:
### Tool to change chrx -> x notation
!samtools view -h $p | sed 's/chr//g' | samtools view -Shb - -o $p2
!samtools index $p2
!samtools view -c $p
!samtools view -c $p2

# 3) Run modern Huggler (Ind 2 from January 2023)

### Optional Pre-Processing:
Change Huggler Y bam to ChrY notation

In [29]:
%%time 

### Tool to change chrx -> x notation
p = "/mnt/archgen/users/hringbauer/data/brienziYcapture/huggler23.bam"
p2 = "/mnt/archgen/users/hringbauer/data/brienziYcapture/huggler23b.bam"

!samtools view -h $p | sed 's/chr//g' | samtools view -Shb - -o $p2

CPU times: user 394 ms, sys: 110 ms, total: 504 ms
Wall time: 19.1 s


In [30]:
%%time 
### Now Index the file
!samtools index $p2

CPU times: user 30.8 ms, sys: 21 ms, total: 51.8 ms
Wall time: 2.06 s


In [31]:
!samtools view -c $p
!samtools view -c $p2

332338
332338


# Do the Y Chromosome Calling

In [86]:
%%time
p2 = "/mnt/archgen/users/hringbauer/data/brienziYcapture/huggler23b.bam"

df_ch, df_der = call_y_bam(df=df, 
                           path_bam=p2)

Average Coverage: 3.2344x
#Sites covered: 1108/73148
#Derived Loci: 
62 / 1108 covered>0
CPU times: user 137 ms, sys: 72.9 ms, total: 210 ms
Wall time: 4.86 s


In [33]:
dft = df_ch[df_ch["Subgroup Name"].str[0]=="G"]
np.sum(dft["alt#"].values) / np.sum(dft["alt#"].values + dft["ref#"].values)

0.00019433324264448676

In [41]:
dft = df_ch[df_ch["Subgroup Name"].str[0]=="R"]
np.sum(dft["alt#"].values) / np.sum(dft["alt#"].values + dft["ref#"].values)

0.1524885804116984

In [34]:
dft = df_ch[df_ch["Subgroup Name"].str.contains("G")]
s = np.sum(dft["alt#"].values) / np.sum(dft["alt#"].values + dft["ref#"].values)
print(f"Derived Frac: {s:.6f}")

cov = np.mean(dft["alt#"].values + dft["ref#"].values)
print(f"Average Coverage: {cov:.2f}x")

Derived Frac: 0.000194
Average Coverage: 219.91x


In [87]:
df_der[-25:]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
37,PF5963,Y,21889767,G,A,P1~_or_K2b2a~,M74;_N12,rs2032635,1,0,0,0,0,1
38,S8,Y,7963031,T,G,P_or_K2b2,P295;_PF5866,rs895530,0,0,45,0,0,45
39,M207,Y,15581983,A,G,R,Page37;_UTY2,NaN,0,0,40,0,0,40
40,PF6112,Y,7570822,G,C,R1,P294,NaN,0,169,0,0,0,169
41,S1,Y,22750583,C,A,R1,M306;_PF6147,NaN,35,0,0,0,0,35
42,Page29,Y,15026424,A,C,R1,M173;_P241,NaN,0,36,0,0,0,36
43,PF6105,Y,21183643,A,G,R1b,L780;_YSC0000254,NaN,0,0,32,0,0,32
44,M343,Y,2887824,C,A,R1b,PF6242,NaN,405,0,0,0,0,405
45,M415,Y,9170545,C,A,R1b,PF6251,NaN,95,0,0,0,0,95
46,L389,Y,28733101,C,G,R1b1a,PF6531,NaN,0,0,25,0,0,25


In [95]:
dft = df_ch[df_ch["Subgroup Name"].str.contains("R1b1a1b1a1a2a")]
dft

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
82,DF17,Y,6631746,T,G,R1b1a1b1a1a2a1a2,S455,NaN,0,1,0,157,157,0
556,S230,Y,15508140,T,G,R1b1a1b1a1a2a1a1,Z209,NaN,0,0,0,14,14,0
620,S356,Y,16310705,G,A,R1b1a1b1a1a2a1a1,Z220,NaN,0,0,159,0,159,0
621,Z262,Y,16320197,C,T,R1b1a1b1a1a2a1b1a,NaN,NaN,0,587,0,0,587,0
708,Z195,Y,17922066,G,A,R1b1a1b1a1a2a1,S227,NaN,0,0,194,0,194,0
710,Z216,Y,18021350,C,T,R1b1a1b1a1a2a1a1a1,NaN,NaN,0,589,0,0,589,0
793,Z224,Y,19179436,T,C,R1b1a1b1a1a2a7~,S453,NaN,0,0,0,143,143,0
920,S348,Y,21827794,A,C,R1b1a1b1a1a2a1a1a1a,Z214,NaN,6,0,0,0,6,0
1024,S229,Y,22293941,C,G,R1b1a1b1a1a2a1a,Z274,NaN,0,5,0,0,5,0
1069,L1246,Y,22974840,A,G,R1b1a1b1a1a2a~,S1285,NaN,9,0,0,0,9,0


In [92]:
df_ch[df_ch["Name"].str.contains("P312")]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
1013,P312,Y,22157311,C,A,R1b1a1b1a1a2,PF6547;_S116,NaN,346,0,1,0,0,346


# 4) Run modern Huggler (Ind 3 from Nov 2023)

### Pre-Processing

In [97]:
%%time 

### Tool to change chrx -> x notation
p = "/mnt/archgen/users/hringbauer/brienzi/data/huggler_y2/s23-14214.bam"
p2 = "/mnt/archgen/users/hringbauer/data/brienziYcapture/huggler23.iid3.bam"

CPU times: user 8 µs, sys: 0 ns, total: 8 µs
Wall time: 15.3 µs


In [ ]:
!samtools view -h $p | sed 's/chr//g' | samtools view -Shb - -o $p2 # Rewrite chrx->x notation

In [24]:
%%time 
### Now Index the file
!samtools index $p2

CPU times: user 24.9 ms, sys: 18 ms, total: 42.8 ms
Wall time: 1.66 s


In [25]:
%%time
!samtools view -c $p
!samtools view -c $p2

242473
242473
CPU times: user 55 ms, sys: 21.1 ms, total: 76.1 ms
Wall time: 3.02 s


### Call Y Chromosome using Harald's tool

In [98]:
%%time

df_ch, df_der = call_y_bam(df=df,  path_bam=p2)

Average Coverage: 1.3888x
#Sites covered: 1053/73148
#Derived Loci: 
55 / 1053 covered>0
CPU times: user 92.1 ms, sys: 0 ns, total: 92.1 ms
Wall time: 1.92 s


In [27]:
df_der[-50:]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
5,L1137,Y,19047091,C,T,A0-T,NaN,NaN,0,0,0,2,0,2
6,L1125,Y,15467768,A,G,A0-T,NaN,NaN,0,0,59,0,0,59
7,L1121,Y,14496448,G,A,A0-T,NaN,NaN,36,0,0,0,0,36
8,L1120,Y,14496439,G,T,A0-T,NaN,NaN,0,0,0,56,0,56
9,P305,Y,2710154,A,G,A1,NaN,NaN,0,0,32,0,0,32
10,V168,Y,17947672,G,A,A1,NaN,NaN,54,0,0,0,0,54
11,V221,Y,7589303,G,T,A1b,NaN,NaN,0,0,0,33,0,33
12,L1053,Y,18759708,C,A,A1b,NaN,NaN,50,0,0,0,0,50
13,M118,Y,21763965,A,T,A1b1b2b1,NaN,NaN,0,0,0,61,0,61
14,M9379,Y,23244049,G,C,BT,NaN,NaN,0,153,0,0,0,153


In [110]:
df_ch[df_ch["Subgroup Name"]=="R1b1a1b1a1a2b3c"] 

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
401,Z144,Y,14544130,T,G,R1b1a1b1a1a2b3c,NaN,NaN,0,0,137,0,0,137
584,S371,Y,16454755,G,T,R1b1a1b1a1a2b3c,PF6578;_Z145,NaN,0,0,0,214,0,214
1051,Z146,Y,23627289,C,T,R1b1a1b1a1a2b3c,PF6584;_S483,NaN,0,0,0,172,0,172


In [107]:
df_ch[df_ch["Subgroup Name"]=="R1b1a1b1a1a2a~"] 

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
1014,L1246,Y,22974840,A,G,R1b1a1b1a1a2a~,S1285,NaN,1,0,0,0,1,0


# C) Run Ganders

## C1) John Gander

In [7]:
%%time
p = "/mnt/archgen/users/hringbauer/brienzi/data/gander_y/IonCode_0104_R_2024_03_27_10_03_43_user_S5XL-0322-182-mt24-004_Auto_user_S5XL-0322-182-mt24-004_452.bam"
p2 = "/mnt/archgen/users/hringbauer/data/brienziYcapture/gander.john.bam" # The rewritten .bam file (in correct chr notation)

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 18.1 µs


### One time transformation

In [8]:
!samtools view -h $p | sed 's/chr//g' | samtools view -Shb - -o $p2 # Rewrite chrx->x notation

In [9]:
%%time
!samtools view -c $p
!samtools view -c $p2

500032
500032
CPU times: user 108 ms, sys: 18.6 ms, total: 127 ms
Wall time: 4.98 s


In [11]:
%%time 
### Now Index the file
!samtools index $p2

CPU times: user 51.2 ms, sys: 21.2 ms, total: 72.4 ms
Wall time: 2.48 s


### Call Y Chromosome using Harad's tool

In [66]:
%%time

df_ch, df_der = call_y_bam(df=df,  path_bam=p2)

Average Coverage: 3.8914x
#Sites covered: 1123/73148
#Derived Loci: 
57 / 1123 covered>0
CPU times: user 81.7 ms, sys: 24.5 ms, total: 106 ms
Wall time: 2.58 s


In [67]:
df_der

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#
0,L1130,Y,16661010,T,G,A0-T,NaN,NaN,0,0,546,0,0,546
1,L1085,Y,2790726,T,C,A0-T,NaN,NaN,0,3,0,0,0,3
2,L1089,Y,2887280,G,C,A0-T,NaN,NaN,0,95,0,0,0,95
3,L1121,Y,14496448,G,A,A0-T,NaN,NaN,3,0,0,0,0,3
4,L1155,Y,22191266,G,C,A0-T,NaN,NaN,0,871,0,0,0,871
5,L1120,Y,14496439,G,T,A0-T,NaN,NaN,0,0,0,15,0,15
6,L1118,Y,14231291,T,C,A0-T,NaN,NaN,0,7,0,0,0,7
7,L1125,Y,15467768,A,G,A0-T,NaN,NaN,0,0,184,0,0,184
8,L1145,Y,21739790,C,T,A0-T,NaN,NaN,0,0,0,737,0,737
9,L1137,Y,19047091,C,T,A0-T,NaN,NaN,0,0,0,18,0,18


In [65]:
df_ch[df_ch["Subgroup Name"]=="R1b1a1b1a1a2h"]

,Name,chrom,pos,ref,alt,Subgroup Name,Alternate Names,rs numbers,A,C,G,T,ref#,alt#


In [ ]:
R1b1a1b1a1a2a1
R1b1a1b1a1a2a1a2

In [ ]:
df_der

# Summary
## Background Info
R1b1a1b1a1a2: 
R1b-P312

Trifurcates into:

- R1b1a1b1a1a2a: DF27 (Brienzi)
- R1b1a1b1a1a2b: U152  (Modern Hugglers, further derived  R1b1a1b1a1a2b3c)
- R1b1a1b1a1a2c1: L21


## Ancient Brienzi:
- Extract 1: R1b1a1b1a1a2a (1 Marker derived, clear path)
- Extract 2: R1b1a1b1a1a2a (1 Marker derived, clear path)

Notably, Z195 (R1b1a1b1a1a2a1) ancestral

## Modern Hugglers
- Individual 1 (2022): R1b1a1b1a1a2b3c (3 SNPs derived)
- Individual 2 (January 2023): R1b1a1b1a1a2b3c (3 SNPs derived)
- Individual 3 (November 2023): R1b1a1b1a1a2b3c (3 SNPs derived)

## Modern Ganders
- John Gander:
Marker U152 ancestral (R1b1a1b1a1a2b, Huggler)
Marker L21 ancestral (R1b1a1b1a1a2c1)

So must be DF27:
Z195 (subgroup of DF27) ancestral (R1b1a1b1a1a2a1)